In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

In [2]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import dlib
import re

In [3]:
import cv2
from PIL import Image

# Define a few variables

In [4]:
basedir = './dataset_AMLS_22-23/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Define a set of functions to be used to extract the features 

In [5]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [6]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

In [7]:
def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

In [8]:
# This function was added by myself
def generate_dict_from_df(df, keys_col, values_col):
    zip_ = zip(df[keys_col], df[values_col])
    l = list(zip_)
    d = dict(l)
    return d

In [9]:
def extract_features_labels():
    """
    This funtion extracts the landmarks features for all images in the folder 'dataset/celeba'.
    It also extract the gender label for each image.
    :return:
        landmark_features:  an array containing 68 landmark points for each image in which a face was detected
        gender_labels:      an array containing the gender label (male=0 and female=1) for each image in
                            which a face was detected
    """
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    target_size = None
    #labels_file = open(os.path.join(basedir, labels_filename), 'r')
    #lines = labels_file.readlines()
    labels_df = pd.read_csv(os.path.join(basedir, labels_filename), sep='\t')
    gender_labels = generate_dict_from_df(labels_df, 'img_name', 'gender')
    #smiling_labels = generate_dict_from_df(labels_df, 'file_name', 'smiling')
    #gender_labels = {line.split(',')[0] : int(line.split(',')[6]) for line in lines[2:]}
    if os.path.isdir(images_dir):
        all_features = []
        all_gender_labels = []
        #all_smiling_labels = []
        image_files = []
        i = 1
        for img_path in image_paths:
            print(f'Iteration {i} out of {len(image_paths)}')
            #file_name= img_path.split('.')[1].split('/')[-1]
            #file_name = str(img_path).replace('\\\\', 'mmmm')
            new_file_name = re.search('img\\\\(.*)', img_path).group(1)
            #new_img_path = re.search('\\(.*).png', img_path)
            #print(file_name)
            #print(new_file_name)

            # load image
            img = tf.keras.utils.img_to_array(
                tf.keras.utils.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_gender_labels.append(gender_labels[new_file_name])
                #all_smiling_labels.append(smiling_labels[new_file_name])
                image_files.append(new_file_name)
            i += 1

    landmark_features = np.array(all_features)
    gender_labels = (np.array(all_gender_labels) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
    #smiling_labels = (np.array(all_smiling_labels) + 1)/2
    return landmark_features, gender_labels, image_files

In [10]:
images_dir = 'dataset_AMLS_22-23/celeba/img/'

data_dir = 'dataset_AMLS_22-23/celeba/'

# Create a dataset containing the mouth landmarks

In [16]:
all_features = []
all_smiling_labels = []
all_images_used = []

In [21]:
# Go through each image and extract the mouth landmarks
for i in range(5000):
    print(f'Processing image {i+1} out of 5000')
    image = Image.open(f'dataset_AMLS_22-23/celeba/img/{i}.jpg')
    image_array = np.array(image)
    features,_ = run_dlib_shape(image_array)
    if features is not None:  # The model detected it as a face
        all_features.append(features[48:])
        all_smiling_labels.append(int(df.iloc[i].smiling))
        all_images_used.append(f'{i}.jpg')
    

Processing image 1 out of 5000
Processing image 2 out of 5000
Processing image 3 out of 5000
Processing image 4 out of 5000
Processing image 5 out of 5000
Processing image 6 out of 5000
Processing image 7 out of 5000
Processing image 8 out of 5000
Processing image 9 out of 5000
Processing image 10 out of 5000
Processing image 11 out of 5000
Processing image 12 out of 5000
Processing image 13 out of 5000
Processing image 14 out of 5000
Processing image 15 out of 5000
Processing image 16 out of 5000
Processing image 17 out of 5000
Processing image 18 out of 5000
Processing image 19 out of 5000
Processing image 20 out of 5000
Processing image 21 out of 5000
Processing image 22 out of 5000
Processing image 23 out of 5000
Processing image 24 out of 5000
Processing image 25 out of 5000
Processing image 26 out of 5000
Processing image 27 out of 5000
Processing image 28 out of 5000
Processing image 29 out of 5000
Processing image 30 out of 5000
Processing image 31 out of 5000
Processing image 

Processing image 255 out of 5000
Processing image 256 out of 5000
Processing image 257 out of 5000
Processing image 258 out of 5000
Processing image 259 out of 5000
Processing image 260 out of 5000
Processing image 261 out of 5000
Processing image 262 out of 5000
Processing image 263 out of 5000
Processing image 264 out of 5000
Processing image 265 out of 5000
Processing image 266 out of 5000
Processing image 267 out of 5000
Processing image 268 out of 5000
Processing image 269 out of 5000
Processing image 270 out of 5000
Processing image 271 out of 5000
Processing image 272 out of 5000
Processing image 273 out of 5000
Processing image 274 out of 5000
Processing image 275 out of 5000
Processing image 276 out of 5000
Processing image 277 out of 5000
Processing image 278 out of 5000
Processing image 279 out of 5000
Processing image 280 out of 5000
Processing image 281 out of 5000
Processing image 282 out of 5000
Processing image 283 out of 5000
Processing image 284 out of 5000
Processing

Processing image 507 out of 5000
Processing image 508 out of 5000
Processing image 509 out of 5000
Processing image 510 out of 5000
Processing image 511 out of 5000
Processing image 512 out of 5000
Processing image 513 out of 5000
Processing image 514 out of 5000
Processing image 515 out of 5000
Processing image 516 out of 5000
Processing image 517 out of 5000
Processing image 518 out of 5000
Processing image 519 out of 5000
Processing image 520 out of 5000
Processing image 521 out of 5000
Processing image 522 out of 5000
Processing image 523 out of 5000
Processing image 524 out of 5000
Processing image 525 out of 5000
Processing image 526 out of 5000
Processing image 527 out of 5000
Processing image 528 out of 5000
Processing image 529 out of 5000
Processing image 530 out of 5000
Processing image 531 out of 5000
Processing image 532 out of 5000
Processing image 533 out of 5000
Processing image 534 out of 5000
Processing image 535 out of 5000
Processing image 536 out of 5000
Processing

Processing image 757 out of 5000
Processing image 758 out of 5000
Processing image 759 out of 5000
Processing image 760 out of 5000
Processing image 761 out of 5000
Processing image 762 out of 5000
Processing image 763 out of 5000
Processing image 764 out of 5000
Processing image 765 out of 5000
Processing image 766 out of 5000
Processing image 767 out of 5000
Processing image 768 out of 5000
Processing image 769 out of 5000
Processing image 770 out of 5000
Processing image 771 out of 5000
Processing image 772 out of 5000
Processing image 773 out of 5000
Processing image 774 out of 5000
Processing image 775 out of 5000
Processing image 776 out of 5000
Processing image 777 out of 5000
Processing image 778 out of 5000
Processing image 779 out of 5000
Processing image 780 out of 5000
Processing image 781 out of 5000
Processing image 782 out of 5000
Processing image 783 out of 5000
Processing image 784 out of 5000
Processing image 785 out of 5000
Processing image 786 out of 5000
Processing

Processing image 1010 out of 5000
Processing image 1011 out of 5000
Processing image 1012 out of 5000
Processing image 1013 out of 5000
Processing image 1014 out of 5000
Processing image 1015 out of 5000
Processing image 1016 out of 5000
Processing image 1017 out of 5000
Processing image 1018 out of 5000
Processing image 1019 out of 5000
Processing image 1020 out of 5000
Processing image 1021 out of 5000
Processing image 1022 out of 5000
Processing image 1023 out of 5000
Processing image 1024 out of 5000
Processing image 1025 out of 5000
Processing image 1026 out of 5000
Processing image 1027 out of 5000
Processing image 1028 out of 5000
Processing image 1029 out of 5000
Processing image 1030 out of 5000
Processing image 1031 out of 5000
Processing image 1032 out of 5000
Processing image 1033 out of 5000
Processing image 1034 out of 5000
Processing image 1035 out of 5000
Processing image 1036 out of 5000
Processing image 1037 out of 5000
Processing image 1038 out of 5000
Processing ima

...

Processing image 3922 out of 5000
Processing image 3923 out of 5000
Processing image 3924 out of 5000
Processing image 3925 out of 5000
Processing image 3926 out of 5000
Processing image 3927 out of 5000
Processing image 3928 out of 5000
Processing image 3929 out of 5000
Processing image 3930 out of 5000
Processing image 3931 out of 5000
Processing image 3932 out of 5000
Processing image 3933 out of 5000
Processing image 3934 out of 5000
Processing image 3935 out of 5000
Processing image 3936 out of 5000
Processing image 3937 out of 5000
Processing image 3938 out of 5000
Processing image 3939 out of 5000
Processing image 3940 out of 5000
Processing image 3941 out of 5000
Processing image 3942 out of 5000
Processing image 3943 out of 5000
Processing image 3944 out of 5000
Processing image 3945 out of 5000
Processing image 3946 out of 5000
Processing image 3947 out of 5000
Processing image 3948 out of 5000
Processing image 3949 out of 5000
Processing image 3950 out of 5000
Processing ima

Processing image 4165 out of 5000
Processing image 4166 out of 5000
Processing image 4167 out of 5000
Processing image 4168 out of 5000
Processing image 4169 out of 5000
Processing image 4170 out of 5000
Processing image 4171 out of 5000
Processing image 4172 out of 5000
Processing image 4173 out of 5000
Processing image 4174 out of 5000
Processing image 4175 out of 5000
Processing image 4176 out of 5000
Processing image 4177 out of 5000
Processing image 4178 out of 5000
Processing image 4179 out of 5000
Processing image 4180 out of 5000
Processing image 4181 out of 5000
Processing image 4182 out of 5000
Processing image 4183 out of 5000
Processing image 4184 out of 5000
Processing image 4185 out of 5000
Processing image 4186 out of 5000
Processing image 4187 out of 5000
Processing image 4188 out of 5000
Processing image 4189 out of 5000
Processing image 4190 out of 5000
Processing image 4191 out of 5000
Processing image 4192 out of 5000
Processing image 4193 out of 5000
Processing ima

Processing image 4409 out of 5000
Processing image 4410 out of 5000
Processing image 4411 out of 5000
Processing image 4412 out of 5000
Processing image 4413 out of 5000
Processing image 4414 out of 5000
Processing image 4415 out of 5000
Processing image 4416 out of 5000
Processing image 4417 out of 5000
Processing image 4418 out of 5000
Processing image 4419 out of 5000
Processing image 4420 out of 5000
Processing image 4421 out of 5000
Processing image 4422 out of 5000
Processing image 4423 out of 5000
Processing image 4424 out of 5000
Processing image 4425 out of 5000
Processing image 4426 out of 5000
Processing image 4427 out of 5000
Processing image 4428 out of 5000
Processing image 4429 out of 5000
Processing image 4430 out of 5000
Processing image 4431 out of 5000
Processing image 4432 out of 5000
Processing image 4433 out of 5000
Processing image 4434 out of 5000
Processing image 4435 out of 5000
Processing image 4436 out of 5000
Processing image 4437 out of 5000
Processing ima

Processing image 4650 out of 5000
Processing image 4651 out of 5000
Processing image 4652 out of 5000
Processing image 4653 out of 5000
Processing image 4654 out of 5000
Processing image 4655 out of 5000
Processing image 4656 out of 5000
Processing image 4657 out of 5000
Processing image 4658 out of 5000
Processing image 4659 out of 5000
Processing image 4660 out of 5000
Processing image 4661 out of 5000
Processing image 4662 out of 5000
Processing image 4663 out of 5000
Processing image 4664 out of 5000
Processing image 4665 out of 5000
Processing image 4666 out of 5000
Processing image 4667 out of 5000
Processing image 4668 out of 5000
Processing image 4669 out of 5000
Processing image 4670 out of 5000
Processing image 4671 out of 5000
Processing image 4672 out of 5000
Processing image 4673 out of 5000
Processing image 4674 out of 5000
Processing image 4675 out of 5000
Processing image 4676 out of 5000
Processing image 4677 out of 5000
Processing image 4678 out of 5000
Processing ima

Processing image 4891 out of 5000
Processing image 4892 out of 5000
Processing image 4893 out of 5000
Processing image 4894 out of 5000
Processing image 4895 out of 5000
Processing image 4896 out of 5000
Processing image 4897 out of 5000
Processing image 4898 out of 5000
Processing image 4899 out of 5000
Processing image 4900 out of 5000
Processing image 4901 out of 5000
Processing image 4902 out of 5000
Processing image 4903 out of 5000
Processing image 4904 out of 5000
Processing image 4905 out of 5000
Processing image 4906 out of 5000
Processing image 4907 out of 5000
Processing image 4908 out of 5000
Processing image 4909 out of 5000
Processing image 4910 out of 5000
Processing image 4911 out of 5000
Processing image 4912 out of 5000
Processing image 4913 out of 5000
Processing image 4914 out of 5000
Processing image 4915 out of 5000
Processing image 4916 out of 5000
Processing image 4917 out of 5000
Processing image 4918 out of 5000
Processing image 4919 out of 5000
Processing ima

### Insert the landmark coordinates into a numpy array

In [54]:
dataset = np.ndarray((4795, 40))

In [58]:
for i in range(len(all_features)):
    print(f'{i} out of 4795')
    dataset[i] = all_features[i].reshape((40,))

0 out of 4795
1 out of 4795
2 out of 4795
3 out of 4795
4 out of 4795
5 out of 4795
6 out of 4795
7 out of 4795
8 out of 4795
9 out of 4795
10 out of 4795
11 out of 4795
12 out of 4795
13 out of 4795
14 out of 4795
15 out of 4795
16 out of 4795
17 out of 4795
18 out of 4795
19 out of 4795
20 out of 4795
21 out of 4795
22 out of 4795
23 out of 4795
24 out of 4795
25 out of 4795
26 out of 4795
27 out of 4795
28 out of 4795
29 out of 4795
30 out of 4795
31 out of 4795
32 out of 4795
33 out of 4795
34 out of 4795
35 out of 4795
36 out of 4795
37 out of 4795
38 out of 4795
39 out of 4795
40 out of 4795
41 out of 4795
42 out of 4795
43 out of 4795
44 out of 4795
45 out of 4795
46 out of 4795
47 out of 4795
48 out of 4795
49 out of 4795
50 out of 4795
51 out of 4795
52 out of 4795
53 out of 4795
54 out of 4795
55 out of 4795
56 out of 4795
57 out of 4795
58 out of 4795
59 out of 4795
60 out of 4795
61 out of 4795
62 out of 4795
63 out of 4795
64 out of 4795
65 out of 4795
66 out of 4795
67 ou

3511 out of 4795
3512 out of 4795
3513 out of 4795
3514 out of 4795
3515 out of 4795
3516 out of 4795
3517 out of 4795
3518 out of 4795
3519 out of 4795
3520 out of 4795
3521 out of 4795
3522 out of 4795
3523 out of 4795
3524 out of 4795
3525 out of 4795
3526 out of 4795
3527 out of 4795
3528 out of 4795
3529 out of 4795
3530 out of 4795
3531 out of 4795
3532 out of 4795
3533 out of 4795
3534 out of 4795
3535 out of 4795
3536 out of 4795
3537 out of 4795
3538 out of 4795
3539 out of 4795
3540 out of 4795
3541 out of 4795
3542 out of 4795
3543 out of 4795
3544 out of 4795
3545 out of 4795
3546 out of 4795
3547 out of 4795
3548 out of 4795
3549 out of 4795
3550 out of 4795
3551 out of 4795
3552 out of 4795
3553 out of 4795
3554 out of 4795
3555 out of 4795
3556 out of 4795
3557 out of 4795
3558 out of 4795
3559 out of 4795
3560 out of 4795
3561 out of 4795
3562 out of 4795
3563 out of 4795
3564 out of 4795
3565 out of 4795
3566 out of 4795
3567 out of 4795
3568 out of 4795
3569 out of 47

In [59]:
dataset

array([[ 72., 152.,  75., ..., 163.,  79., 162.],
       [ 68., 151.,  73., ..., 162.,  79., 161.],
       [ 72., 154.,  77., ..., 148.,  83., 147.],
       ...,
       [ 71., 152.,  77., ..., 158.,  82., 157.],
       [ 69., 152.,  78., ..., 155.,  86., 154.],
       [ 74., 150.,  75., ..., 161.,  80., 159.]])

In [ ]:
all_smiling_labels = [int((element+1)/2) for element in all_smiling_labels]

In [66]:
labels = np.array(all_smiling_labels)

In [72]:
labels

array([1., 1., 0., ..., 1., 1., 1.])

In [73]:
training_features = dataset[:4000]; testing_features = dataset[4000:];
training_labels = labels[:4000]; testing_labels = labels[4000:];

In [28]:
def obtain_distances_matrix(features):
    distances_matrix = np.ndarray((20, 20))
    for i in range(len(features)):
        for j in range(len(features)):
            distances_matrix[i][j] = np.linalg.norm(features[i] - features[j])
    return distances_matrix

In [29]:
dataset = np.ndarray((len(all_features), 190))

In [30]:
for i in range(len(all_features)):
    print(f'Completing image {i+1} out of {len(all_features)}')
    distances_matrix = obtain_distances_matrix(all_features[i])  # Obtain upper triangle
    D = distances_matrix[np.triu_indices(20, 1)]
    D_l1 = sum(D)
    F = D / D_l1
    dataset[i] = F

Completing image 1 out of 4795
Completing image 2 out of 4795
Completing image 3 out of 4795
Completing image 4 out of 4795
Completing image 5 out of 4795
Completing image 6 out of 4795
Completing image 7 out of 4795
Completing image 8 out of 4795
Completing image 9 out of 4795
Completing image 10 out of 4795
Completing image 11 out of 4795
Completing image 12 out of 4795
Completing image 13 out of 4795
Completing image 14 out of 4795
Completing image 15 out of 4795
Completing image 16 out of 4795
Completing image 17 out of 4795
Completing image 18 out of 4795
Completing image 19 out of 4795
Completing image 20 out of 4795
Completing image 21 out of 4795
Completing image 22 out of 4795
Completing image 23 out of 4795
Completing image 24 out of 4795
Completing image 25 out of 4795
Completing image 26 out of 4795
Completing image 27 out of 4795
Completing image 28 out of 4795
Completing image 29 out of 4795
Completing image 30 out of 4795
Completing image 31 out of 4795
Completing image 

Completing image 257 out of 4795
Completing image 258 out of 4795
Completing image 259 out of 4795
Completing image 260 out of 4795
Completing image 261 out of 4795
Completing image 262 out of 4795
Completing image 263 out of 4795
Completing image 264 out of 4795
Completing image 265 out of 4795
Completing image 266 out of 4795
Completing image 267 out of 4795
Completing image 268 out of 4795
Completing image 269 out of 4795
Completing image 270 out of 4795
Completing image 271 out of 4795
Completing image 272 out of 4795
Completing image 273 out of 4795
Completing image 274 out of 4795
Completing image 275 out of 4795
Completing image 276 out of 4795
Completing image 277 out of 4795
Completing image 278 out of 4795
Completing image 279 out of 4795
Completing image 280 out of 4795
Completing image 281 out of 4795
Completing image 282 out of 4795
Completing image 283 out of 4795
Completing image 284 out of 4795
Completing image 285 out of 4795
Completing image 286 out of 4795
Completing

Completing image 520 out of 4795
Completing image 521 out of 4795
Completing image 522 out of 4795
Completing image 523 out of 4795
Completing image 524 out of 4795
Completing image 525 out of 4795
Completing image 526 out of 4795
Completing image 527 out of 4795
Completing image 528 out of 4795
Completing image 529 out of 4795
Completing image 530 out of 4795
Completing image 531 out of 4795
Completing image 532 out of 4795
Completing image 533 out of 4795
Completing image 534 out of 4795
Completing image 535 out of 4795
Completing image 536 out of 4795
Completing image 537 out of 4795
Completing image 538 out of 4795
Completing image 539 out of 4795
Completing image 540 out of 4795
Completing image 541 out of 4795
Completing image 542 out of 4795
Completing image 543 out of 4795
Completing image 544 out of 4795
Completing image 545 out of 4795
Completing image 546 out of 4795
Completing image 547 out of 4795
Completing image 548 out of 4795
Completing image 549 out of 4795
Completing

Completing image 777 out of 4795
Completing image 778 out of 4795
Completing image 779 out of 4795
Completing image 780 out of 4795
Completing image 781 out of 4795
Completing image 782 out of 4795
Completing image 783 out of 4795
Completing image 784 out of 4795
Completing image 785 out of 4795
Completing image 786 out of 4795
Completing image 787 out of 4795
Completing image 788 out of 4795
Completing image 789 out of 4795
Completing image 790 out of 4795
Completing image 791 out of 4795
Completing image 792 out of 4795
Completing image 793 out of 4795
Completing image 794 out of 4795
Completing image 795 out of 4795
Completing image 796 out of 4795
Completing image 797 out of 4795
Completing image 798 out of 4795
Completing image 799 out of 4795
Completing image 800 out of 4795
Completing image 801 out of 4795
Completing image 802 out of 4795
Completing image 803 out of 4795
Completing image 804 out of 4795
Completing image 805 out of 4795
Completing image 806 out of 4795
Completing

Completing image 1036 out of 4795
Completing image 1037 out of 4795
Completing image 1038 out of 4795
Completing image 1039 out of 4795
Completing image 1040 out of 4795
Completing image 1041 out of 4795
Completing image 1042 out of 4795
Completing image 1043 out of 4795
Completing image 1044 out of 4795
Completing image 1045 out of 4795
Completing image 1046 out of 4795
Completing image 1047 out of 4795
Completing image 1048 out of 4795
Completing image 1049 out of 4795
Completing image 1050 out of 4795
Completing image 1051 out of 4795
Completing image 1052 out of 4795
Completing image 1053 out of 4795
Completing image 1054 out of 4795
Completing image 1055 out of 4795
Completing image 1056 out of 4795
Completing image 1057 out of 4795
Completing image 1058 out of 4795
Completing image 1059 out of 4795
Completing image 1060 out of 4795
Completing image 1061 out of 4795
Completing image 1062 out of 4795
Completing image 1063 out of 4795
Completing image 1064 out of 4795
Completing ima

...

Completing image 3616 out of 4795
Completing image 3617 out of 4795
Completing image 3618 out of 4795
Completing image 3619 out of 4795
Completing image 3620 out of 4795
Completing image 3621 out of 4795
Completing image 3622 out of 4795
Completing image 3623 out of 4795
Completing image 3624 out of 4795
Completing image 3625 out of 4795
Completing image 3626 out of 4795
Completing image 3627 out of 4795
Completing image 3628 out of 4795
Completing image 3629 out of 4795
Completing image 3630 out of 4795
Completing image 3631 out of 4795
Completing image 3632 out of 4795
Completing image 3633 out of 4795
Completing image 3634 out of 4795
Completing image 3635 out of 4795
Completing image 3636 out of 4795
Completing image 3637 out of 4795
Completing image 3638 out of 4795
Completing image 3639 out of 4795
Completing image 3640 out of 4795
Completing image 3641 out of 4795
Completing image 3642 out of 4795
Completing image 3643 out of 4795
Completing image 3644 out of 4795
Completing ima

Completing image 3865 out of 4795
Completing image 3866 out of 4795
Completing image 3867 out of 4795
Completing image 3868 out of 4795
Completing image 3869 out of 4795
Completing image 3870 out of 4795
Completing image 3871 out of 4795
Completing image 3872 out of 4795
Completing image 3873 out of 4795
Completing image 3874 out of 4795
Completing image 3875 out of 4795
Completing image 3876 out of 4795
Completing image 3877 out of 4795
Completing image 3878 out of 4795
Completing image 3879 out of 4795
Completing image 3880 out of 4795
Completing image 3881 out of 4795
Completing image 3882 out of 4795
Completing image 3883 out of 4795
Completing image 3884 out of 4795
Completing image 3885 out of 4795
Completing image 3886 out of 4795
Completing image 3887 out of 4795
Completing image 3888 out of 4795
Completing image 3889 out of 4795
Completing image 3890 out of 4795
Completing image 3891 out of 4795
Completing image 3892 out of 4795
Completing image 3893 out of 4795
Completing ima

Completing image 4130 out of 4795
Completing image 4131 out of 4795
Completing image 4132 out of 4795
Completing image 4133 out of 4795
Completing image 4134 out of 4795
Completing image 4135 out of 4795
Completing image 4136 out of 4795
Completing image 4137 out of 4795
Completing image 4138 out of 4795
Completing image 4139 out of 4795
Completing image 4140 out of 4795
Completing image 4141 out of 4795
Completing image 4142 out of 4795
Completing image 4143 out of 4795
Completing image 4144 out of 4795
Completing image 4145 out of 4795
Completing image 4146 out of 4795
Completing image 4147 out of 4795
Completing image 4148 out of 4795
Completing image 4149 out of 4795
Completing image 4150 out of 4795
Completing image 4151 out of 4795
Completing image 4152 out of 4795
Completing image 4153 out of 4795
Completing image 4154 out of 4795
Completing image 4155 out of 4795
Completing image 4156 out of 4795
Completing image 4157 out of 4795
Completing image 4158 out of 4795
Completing ima

Completing image 4399 out of 4795
Completing image 4400 out of 4795
Completing image 4401 out of 4795
Completing image 4402 out of 4795
Completing image 4403 out of 4795
Completing image 4404 out of 4795
Completing image 4405 out of 4795
Completing image 4406 out of 4795
Completing image 4407 out of 4795
Completing image 4408 out of 4795
Completing image 4409 out of 4795
Completing image 4410 out of 4795
Completing image 4411 out of 4795
Completing image 4412 out of 4795
Completing image 4413 out of 4795
Completing image 4414 out of 4795
Completing image 4415 out of 4795
Completing image 4416 out of 4795
Completing image 4417 out of 4795
Completing image 4418 out of 4795
Completing image 4419 out of 4795
Completing image 4420 out of 4795
Completing image 4421 out of 4795
Completing image 4422 out of 4795
Completing image 4423 out of 4795
Completing image 4424 out of 4795
Completing image 4425 out of 4795
Completing image 4426 out of 4795
Completing image 4427 out of 4795
Completing ima

Completing image 4663 out of 4795
Completing image 4664 out of 4795
Completing image 4665 out of 4795
Completing image 4666 out of 4795
Completing image 4667 out of 4795
Completing image 4668 out of 4795
Completing image 4669 out of 4795
Completing image 4670 out of 4795
Completing image 4671 out of 4795
Completing image 4672 out of 4795
Completing image 4673 out of 4795
Completing image 4674 out of 4795
Completing image 4675 out of 4795
Completing image 4676 out of 4795
Completing image 4677 out of 4795
Completing image 4678 out of 4795
Completing image 4679 out of 4795
Completing image 4680 out of 4795
Completing image 4681 out of 4795
Completing image 4682 out of 4795
Completing image 4683 out of 4795
Completing image 4684 out of 4795
Completing image 4685 out of 4795
Completing image 4686 out of 4795
Completing image 4687 out of 4795
Completing image 4688 out of 4795
Completing image 4689 out of 4795
Completing image 4690 out of 4795
Completing image 4691 out of 4795
Completing ima

In [31]:
dataset

array([[0.00106039, 0.00249936, 0.00395184, ..., 0.00180232, 0.00325877,
        0.00145737],
       [0.0017046 , 0.00353898, 0.00474805, ..., 0.00189922, 0.00318115,
        0.00130511],
       [0.00447176, 0.00776595, 0.00848458, ..., 0.00203944, 0.0035997 ,
        0.00164911],
       ...,
       [0.00231499, 0.00424414, 0.00618534, ..., 0.00231499, 0.00426164,
        0.00196736],
       [0.00355747, 0.00673128, 0.00869603, ..., 0.00201551, 0.00395274,
        0.00201551],
       [0.0005282 , 0.00217781, 0.00367846, ..., 0.00186745, 0.00344342,
        0.0016703 ]])

In [33]:
labels = np.array(all_smiling_labels)

In [35]:
labels = [int((element+1)/2) for element in labels]

In [37]:
labels = np.array(labels)

In [38]:
labels

array([1, 1, 0, ..., 1, 1, 1])

In [39]:
training_features = dataset[:4000]; testing_features = dataset[4000:];
training_labels = labels[:4000]; testing_labels = labels[4000:];